In [ ]:
from typing import Callable

import polars as pl
import matplotlib.pyplot as plt
import blitzbeaver as bb
import json

In [ ]:
csv_path = "../../aptihramy/data/csv_cleaned"
beaver_folder_path = "../data/beaver_files"
json_folder_path = "../data/json_files"

In [ ]:
class MetricsResult:
    def __init__(
        self,
        verifier: dict[str, float],
        memory_distance: dict[str, float],
        start_years: list[int],
        end_years: list[int],
        chain_lengths: list[int],
        avg_records_match: float,
        avg_trackers_match: float,
        histogram_records: list[int],
        histogram_trackers: list[int],
    ) -> None:
        self.verifier = verifier
        self.memory_distance = memory_distance
        self.start_years = start_years
        self.end_years = end_years
        self.chain_lengths = chain_lengths
        self.avg_records_match = avg_records_match
        self.avg_trackers_match = avg_trackers_match
        self.histogram_records = histogram_records
        self.histogram_trackers = histogram_trackers


class StatClass:
    def __init__(
        self,
        name: str,
        x: list[float],
        y: list[float],
        configs: list[bb.TrackingConfig],
    ) -> None:
        self.name = name
        self.x = x
        self.y = y
        self.configs = configs

In [ ]:
def load_metrics_results(filename: str) -> list[tuple[bb.TrackingConfig, MetricsResult]]:
    filepath = f"{json_folder_path}/{filename}"
    with open(filepath, "r") as file:
        data = json.load(file)
    results = []
    for raw in data:
        config = bb.deserialize_tracking_config(raw["config"])
        data = raw["data"]
        metrics_result = MetricsResult(
            verifier=data["verifier"],
            memory_distance=data["memory_distance"],
            start_years=data["start_years"],
            end_years=data["end_years"],
            chain_lengths=data["chain_lengths"],
            avg_records_match=data["avg_records_match"],
            avg_trackers_match=data["avg_trackers_match"],
            histogram_records=data["histogram_records"],
            histogram_trackers=data["histogram_trackers"],
        )
        results.append((config, metrics_result))
    
    return results

In [ ]:
def histogram_mean(histogram: list[int]) -> float:
    total = 0
    for i, count in enumerate(histogram):
        total += i * count
    return total / sum(histogram)

def histogram_median(histogram: list[int]) -> float:
    n = sum(histogram)
    half = n // 2
    i = 0
    while n > half:
        n -= histogram[i]
        i += 1
    return i - 1

In [ ]:
results = []
# results += load_metrics_results("configs_p1.json")
# results += load_metrics_results("configs_p2.json")
# results += load_metrics_results("configs_p3.json")
# results += load_metrics_results("configs_p4.json")
# results += load_metrics_results("configs_p5.json")
results += load_metrics_results("configs_p6.json")

In [ ]:
def compute_stat_classes(
    results: list[tuple[bb.TrackingConfig, MetricsResult]],
    groupby: Callable[[bb.TrackingConfig], str],
    max_x: int | None = None,
) -> list[StatClass]:

    groups: dict[str, StatClass] = {}

    for result in results:
        config = result[0]
        metrics = result[1]
        x = histogram_mean(metrics.histogram_records)
        y = histogram_mean(metrics.chain_lengths)

        if max_x is not None and x >= max_x:
            continue

        name = groupby(config)

        if not name in groups:
            groups[name] = StatClass(name, [], [], [])
        group = groups[name]

        group.x.append(x)
        group.y.append(y)
        group.configs.append(config)

    return list(groups.values())

def display_stat_classes(stat_classes: list[StatClass]) -> None:
    plt.figure(figsize=(22, 14))
    for stat_class in stat_classes:
        plt.scatter(stat_class.x, stat_class.y, label=stat_class.name, alpha=1.0, s=50)
    plt.legend()
    plt.plot()

In [ ]:
groupby_threshold = lambda c: str(c.tracker.interest_threshold)
groupby_lv_substring = lambda c: str(c.distance_metric.lv_substring_weight if c.distance_metric.lv_substring_weight else "other")
groupby_weights = lambda c: ", ".join(f"{w:.2f}" for w in c.tracker.record_scorer.weights) if c.tracker.record_scorer.weights else "average"
groupby_memory = lambda c: str(c.tracker.memories[0].memory_strategy)
groupby_min_weight_ratio = lambda c: str(c.tracker.record_scorer.min_weight_ratio) if c.tracker.record_scorer.min_weight_ratio else "1.0"
groupby_multiword_threshold = lambda c: str(c.tracker.memories[-1].multiword_threshold_match)

In [ ]:
classes = compute_stat_classes(
    results,
    groupby=groupby_weights,
    max_x=1.3,
)

In [ ]:
display_stat_classes(classes)

In [ ]:
display_stat_classes(classes)

In [ ]:
def find_datapoints(
        classes: list[StatClass],
        x_range: tuple[float, float],
        y_range: tuple[float, float],
) -> list[tuple[float, float, bb.TrackingConfig]]:
    configs = []
    for stat_class in classes:
        for x, y, config in zip(stat_class.x, stat_class.y, stat_class.configs):
            if x_range[0] <= x <= x_range[1] and y_range[0] <= y <= y_range[1]:
                configs.append((x, y, config))
    return configs

In [ ]:
dps = find_datapoints(
    classes,
    x_range=(0.95, 1.05),
    y_range=(2.35, 10.0),
)

In [ ]:
dps

In [ ]:
bb.serialize_tracking_config(dps[-2][2])